In [1]:
import pandas as pd
df_ml = pd.read_csv("../data/processed/ML_Chile_2022-12-27.csv")
df_ml.dropna(subset=["tweet_text"], inplace=True)
# df_f = pd.read_csv("../data/processed/FalabellaAyuda_2022-12-27.csv")
# df_r = pd.read_csv("../data/processed/RipleyChile_2022-12-27.csv")
# df_p = pd.read_csv("../data/processed/tiendas_paris_2022-12-27.csv")

In [2]:
def df_to_list(df):
    texts = df["tweet_text"]
    data = []
    for text in texts:
        data.append(text)
    return data

In [3]:
import numpy as np
import torch
from transformers import BertModel, BertTokenizer
tokenizer_español = BertTokenizer.from_pretrained("../pytorch/", do_lower_case=False)
model = BertModel.from_pretrained("../pytorch")

Some weights of the model checkpoint at ../pytorch were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
from src import Features
inputs = df_to_list(df_ml)
inputs = Features.bert_encoder(inputs, tokenizer_español)
outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state
data_ml = {
        "max_polling_std": Features.std_normalization(Features.max_polling(last_hidden_states)),
        "max_polling_z_score": Features.z_score_normalization(Features.max_polling(last_hidden_states)),
        "max_polling_max": Features.max_min_normalization(Features.max_polling(last_hidden_states)),
        "mean_polling_std": Features.std_normalization(Features.mean_polling(last_hidden_states)),
        "mean_polling_z_score": Features.z_score_normalization(Features.mean_polling(last_hidden_states)),
        "mean_polling_max": Features.max_min_normalization(Features.mean_polling(last_hidden_states)),
    }

In [5]:
def find_indexs(list, value):
    list_aux = list.copy()
    output = []
    while True:
        try:
            index = list_aux.index(value)
            output.append(index)
            list_aux[index] = None
        except ValueError:
            break
    return output

# agrupamiento

# k-means

In [6]:
# k-means
from sklearn.cluster import KMeans
from sklearn import metrics
agrupamiento = {}
metric = []
for i in range(100):
    metrica = []
    for key, value in data_ml.items():
        kmeans = KMeans(
            n_clusters=4,
            init="random",
            max_iter=100,
            random_state=i,
            n_init=1,
        ).fit(value)
        agrupamiento[key] = list(kmeans.labels_)
        metrica.append(metrics.silhouette_score(
            value, kmeans.labels_, metric="sqeuclidean"))
    metric.append(metrica)
    #print(f"{round(metrica[0], 3)} {round(metrica[1], 3)} {round(metrica[2], 3)} {round(metrica[3], 3)} {round(metrica[4], 3)} {round(metrica[5], 3)}")
rs = []
for i in range(6):
    list_aux = []
    for metrica in metric:
        list_aux.append(metrica[i])
    max_ = max(list_aux)
    index = list_aux.index(max_)
    rs.append([index, round(max_, 3)])
rs

[[64, 0.537], [54, 0.121], [1, 0.532], [84, 0.218], [54, 0.288], [60, 0.272]]

In [7]:
i=0
for key, value in data_ml.items():
    kmeans = KMeans(
        n_clusters=4,
        init="random",
        max_iter=100,
        random_state=rs[i][0],
        n_init=1,
    ).fit(value)
    agrupamiento[key] = list(kmeans.labels_)
    i += 1
clusters = {}
for key, value in agrupamiento.items():
    clusters[key] = {
    "cluster0": df_ml.iloc[find_indexs(agrupamiento[key], 0)],
    "cluster1": df_ml.iloc[find_indexs(agrupamiento[key], 1)],
    "cluster2": df_ml.iloc[find_indexs(agrupamiento[key], 2)],
    "cluster3": df_ml.iloc[find_indexs(agrupamiento[key], 3)],
}
# preprocesado vs sentimiento.
print("rs rsScore cluster 1 cluster 2 cluster 3 cluster 4")
i=0
for key0, value0 in clusters.items():
    data = []
    for key1, value1 in value0.items():
        sentiment = dict(value1["sentiment"].value_counts())
        sentiment_values = list(sentiment.values())
        sentiment_labels = list(sentiment.keys())
        sentiment = [f"{label} ({value})" for label, value in zip(sentiment_labels, sentiment_values)]
        data.append(sentiment)
    string = f"{rs[i][0]} & {rs[i][1]} & {data[0]} & {data[1]} & {data[2]} & {data[3]}"
    string = string.replace('[', '')
    string = string.replace(']', '')
    string = string.replace("'", "")
    print(string)
    i += 1

rs rsScore cluster 1 cluster 2 cluster 3 cluster 4
64 & 0.537 & sadness (3), others (1) & others (1) & others (4), joy (1) & others (43), anger (16), sadness (8)
54 & 0.121 & others (5), sadness (3), anger (1) & sadness (2), others (1) & joy (1), others (1) & others (42), anger (15), sadness (6)
1 & 0.532 & others (2), anger (1), joy (1) & others (6), sadness (5) & others (19), anger (10), sadness (2) & others (22), anger (5), sadness (4)
84 & 0.218 & others (7), anger (5), sadness (1) & others (2), anger (1) & sadness (5), others (3), joy (1) & others (37), anger (10), sadness (5)
54 & 0.288 & sadness (3), others (2) & sadness (2), others (1) & joy (1) & others (46), anger (16), sadness (6)
60 & 0.272 & others (4), anger (2), sadness (1), joy (1) & others (11), sadness (6), anger (4) & others (33), anger (10), sadness (4) & others (1)


# k-means++

In [8]:
agrupamiento = {}
metric = []
for i in range(100):
    metrica = []
    for key, value in data_ml.items():
        kmeans = KMeans(
            n_clusters=4,
            init="k-means++",
            max_iter=100,
            random_state=i,
            n_init=1,
        ).fit(value)
        agrupamiento[key] = list(kmeans.labels_)
        metrica.append(metrics.silhouette_score(
            value, kmeans.labels_, metric="sqeuclidean"))
    metric.append(metrica)
    #print(f"{round(metrica[0], 3)} {round(metrica[1], 3)} {round(metrica[2], 3)} {round(metrica[3], 3)} {round(metrica[4], 3)} {round(metrica[5], 3)}")
rs = []
for i in range(6):
    list_aux = []
    for metrica in metric:
        list_aux.append(metrica[i])
    max_ = max(list_aux)
    index = list_aux.index(max_)
    rs.append([index, round(max_, 3)])
rs


[[71, 0.627], [7, 0.107], [3, 0.532], [72, 0.223], [18, 0.229], [90, 0.264]]

In [9]:
i=0
for key, value in data_ml.items():
    kmeans = KMeans(
        n_clusters=4,
        init="k-means++",
        max_iter=100,
        random_state=rs[i][0],
        n_init=1,
    ).fit(value)
    agrupamiento[key] = list(kmeans.labels_)
    i += 1
clusters = {}
for key, value in agrupamiento.items():
    clusters[key] = {
    "cluster0": df_ml.iloc[find_indexs(agrupamiento[key], 0)],
    "cluster1": df_ml.iloc[find_indexs(agrupamiento[key], 1)],
    "cluster2": df_ml.iloc[find_indexs(agrupamiento[key], 2)],
    "cluster3": df_ml.iloc[find_indexs(agrupamiento[key], 3)],
}
# preprocesado vs sentimiento.
print("rs rsScore cluster 1 cluster 2 cluster 3 cluster 4")
i = 0
for key0, value0 in clusters.items():
    data = []
    for key1, value1 in value0.items():
        sentiment = dict(value1["sentiment"].value_counts())
        sentiment_values = list(sentiment.values())
        sentiment_labels = list(sentiment.keys())
        sentiment = [f"{label} ({value})" for label, value in zip(sentiment_labels, sentiment_values)]
        data.append(sentiment)
    string = f"{rs[i][0]} & {rs[i][1]} & {data[0]} & {data[1]} & {data[2]} & {data[3]}"
    string = string.replace('[', '')
    string = string.replace(']', '')
    string = string.replace("'", "")
    print(string)
    i += 1

rs rsScore cluster 1 cluster 2 cluster 3 cluster 4
71 & 0.627 & joy (1), others (1) & others (47), anger (16), sadness (9) & others (1) & sadness (2)
7 & 0.107 & others (41), anger (13), sadness (5) & others (3), sadness (1) & sadness (4), others (1) & others (4), anger (3), sadness (1), joy (1)
3 & 0.532 & others (19), anger (10), sadness (2) & others (22), anger (5), sadness (4) & others (6), sadness (5) & others (2), anger (1), joy (1)
72 & 0.223 & sadness (5), others (2), joy (1) & others (4), anger (1) & others (38), anger (10), sadness (5) & others (5), anger (5), sadness (1)
18 & 0.229 & others (4), joy (1), anger (1) & others (40), anger (13), sadness (5) & others (3), anger (2), sadness (1) & sadness (5), others (2)
90 & 0.264 & others (22), anger (9), sadness (1) & others (3), sadness (1), joy (1) & sadness (6), others (3) & others (21), anger (7), sadness (3)


# fuzzy c-means

In [10]:
from fcmeans import FCM
agrupamiento = {}
metric = []

rs = []
for key, value in data_ml.items():
    data = np.array(value)
    fcm = FCM(n_clusters=4)
    fcm.fit(data)
    labels = fcm.predict(data)
    grupos = fcm.soft_predict(data)
    agrupamiento[key] = labels
    rs.append(round(metrics.silhouette_score(
        value, labels, metric="sqeuclidean"), 3))
    #print(f"{round(metrica[0], 3)} {round(metrica[1], 3)} {round(metrica[2], 3)} {round(metrica[3], 3)} {round(metrica[4], 3)} {round(metrica[5], 3)}")
rs


[0.256, 0.018, 0.429, 0.167, 0.174, 0.128]

In [11]:
clusters = {}
for key, value in agrupamiento.items():
    clusters[key] = {
    "cluster0": df_ml.iloc[find_indexs(list(agrupamiento[key]), 0)],
    "cluster1": df_ml.iloc[find_indexs(list(agrupamiento[key]), 1)],
    "cluster2": df_ml.iloc[find_indexs(list(agrupamiento[key]), 2)],
    "cluster3": df_ml.iloc[find_indexs(list(agrupamiento[key]), 3)],

}
# preprocesado vs sentimiento.
print("rsScore cluster 1 cluster 2 cluster 3 cluster 4")
i = 0
for key0, value0 in clusters.items():
    data = []
    for key1, value1 in value0.items():
        sentiment = dict(value1["sentiment"].value_counts())
        sentiment_values = list(sentiment.values())
        sentiment_labels = list(sentiment.keys())
        sentiment = [f"{label} ({value})" for label, value in zip(sentiment_labels, sentiment_values)]
        data.append(sentiment)
    string = f"{rs[i]} & {data[0]} & {data[1]} & {data[2]} & {data[3]}"
    string = string.replace('[', '')
    string = string.replace(']', '')
    string = string.replace("'", "")
    print(string)
    i += 1

rsScore cluster 1 cluster 2 cluster 3 cluster 4
0.256 &  & others (16), sadness (6), anger (5), joy (1) & others (33), anger (11), sadness (5) & 
0.018 & anger (1) &  & others (29), anger (10), sadness (5) & others (20), sadness (6), anger (5), joy (1)
0.429 & others (16), anger (6), sadness (2) & others (6), sadness (5) & others (10), anger (3), sadness (2), joy (1) & others (17), anger (7), sadness (2)
0.167 & others (28), anger (9), sadness (4) &  & others (21), sadness (7), anger (7), joy (1) & 
0.174 & others (30), anger (9), sadness (3) &  & others (19), sadness (8), anger (7), joy (1) & 
0.128 &  & others (14), sadness (6), anger (4), joy (1) & others (22), anger (7), sadness (3) & others (13), anger (5), sadness (2)


# eugenspace fuzzy c-means

In [12]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=3, n_iter=2)

agrupamiento = {}
metric = []
rs = []
for key, value in data_ml.items():
    data = svd.fit_transform(np.array(value))
    fcm = FCM(n_clusters=4)
    fcm.fit(data)
    labels = fcm.predict(data)
    grupos = fcm.soft_predict(data)
    agrupamiento[key] = labels
    rs.append(round(metrics.silhouette_score(
        value, labels, metric="sqeuclidean"), 3))
    #print(f"{round(metrica[0], 3)} {round(metrica[1], 3)} {round(metrica[2], 3)} {round(metrica[3], 3)} {round(metrica[4], 3)} {round(metrica[5], 3)}")
rs

[0.094, 0.035, 0.509, 0.092, 0.104, 0.151]

In [13]:
clusters = {}
for key, value in agrupamiento.items():
    clusters[key] = {
    "cluster0": df_ml.iloc[find_indexs(list(agrupamiento[key]), 0)],
    "cluster1": df_ml.iloc[find_indexs(list(agrupamiento[key]), 1)],
    "cluster2": df_ml.iloc[find_indexs(list(agrupamiento[key]), 2)],
    "cluster3": df_ml.iloc[find_indexs(list(agrupamiento[key]), 3)],

}
# preprocesado vs sentimiento.
print("rsScore cluster 1 cluster 2 cluster 3 cluster 4")
i = 0
for key0, value0 in clusters.items():
    data = []
    for key1, value1 in value0.items():
        sentiment = dict(value1["sentiment"].value_counts())
        sentiment_values = list(sentiment.values())
        sentiment_labels = list(sentiment.keys())
        sentiment = [f"{label} ({value})" for label, value in zip(sentiment_labels, sentiment_values)]
        data.append(sentiment)
    string = f"{rs[i]} & {data[0]} & {data[1]} & {data[2]} & {data[3]}"
    string = string.replace('[', '')
    string = string.replace(']', '')
    string = string.replace("'", "")
    print(string)
    i += 1

rsScore cluster 1 cluster 2 cluster 3 cluster 4
0.094 & others (24), anger (9), sadness (6) & sadness (2) & others (5), joy (1) & others (20), anger (7), sadness (3)
0.035 & others (19), anger (7), sadness (2) & sadness (3), others (2), joy (1) & others (17), anger (6), sadness (1) & others (11), sadness (5), anger (3)
0.509 & others (17), anger (8), sadness (2) & others (4), anger (2), joy (1) & others (22), anger (6), sadness (4) & others (6), sadness (5)
0.092 & others (9), anger (7), sadness (1), joy (1) & others (16), anger (3), sadness (1) & sadness (5), others (2) & others (22), anger (6), sadness (4)
0.104 & others (16), anger (6), sadness (2) & sadness (4), others (2) & others (7), sadness (2), anger (2), joy (1) & others (24), anger (8), sadness (3)
0.151 & sadness (5), others (5), joy (1) & others (13), anger (5), sadness (2) & others (11), anger (5), sadness (2) & others (20), anger (6), sadness (2)


# Extracción de infrmación

In [14]:
clusters["max_polling_std"]["cluster0"]

,user_id,user_location,created_at,tweet_text,@usernames_in_tweet,hashtags_in_tweet,sentiment,score
2,211381686,NaN,2022-12-26 20:40:29+00:00,hola! tengo la cuenta bloqueada hace varios di...,['ML_Chile'],NaN,sadness,0.500699
3,1607430801557258242,NaN,2022-12-26 17:44:02+00:00,tengo un problema para entrar a mi cuenta lo q...,['ML_Chile'],NaN,anger,0.394867
4,353298514,NaN,2022-12-26 17:35:24+00:00,"favor su ayuda, me hicieron un pago duplicado ...",['ML_Chile'],NaN,sadness,0.337873
5,1512164712,Santiago - Chile,2022-12-26 16:54:40+00:00,hola necesito poder contactarme con ustedes y ...,['ML_Chile'],NaN,sadness,0.384147
6,1298637472923037696,NaN,2022-12-26 12:55:52+00:00,"el ladron de su repartidor se robo mi pedido, ...",['ML_Chile'],NaN,anger,0.613460
7,1298637472923037696,NaN,2022-12-26 12:53:17+00:00,compre un regalo de navidad para mi novia por ...,['ML_Chile'],NaN,anger,0.757257
9,267489300,"San Felipe, Chile",2022-12-23 23:31:33+00:00,dicen ser serios y puntuales con los envios ll...,['ML_Chile'],NaN,anger,0.496347
17,1380617915708403715,"Coyhaique, Aysén del General C",2022-12-23 17:52:45+00:00,porque no dicen que esta promocionando llega s...,['ML_Chile'],NaN,others,0.482545
21,1188248605150400512,"Metropolitana de Santiago, Chi",2022-12-23 15:33:19+00:00,mercadolibre me suspendio mi cuenta por no man...,['ML_Chile'],NaN,anger,0.547481
24,171369500,NaN,2022-12-23 12:14:58+00:00,hice una compra en modalidad full de 4 product...,['ML_Chile'],NaN,anger,0.626194


In [24]:
import openai
import os
import ast
from config import OPENAI_API_KEY

def ei(context, t):
    response = openai.Completion.create(
            model="text-davinci-003",
            prompt=f"{context}:{t!r}",
            temperature=0.7,
            max_tokens=64,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0
        )
    return response


openai.api_key = OPENAI_API_KEY
context_e = """
        Extrae eventos especificando el tipo de evento,
        tipos de evento:[Fecha, lugar, concecuencias],
        el resultado retornalo como un diccionario de python, ejemplo:
        {
          "tipo evento": "algo",
          "lugar": "algo",
          "consecuencias": "algo",
        }
        es imperativo que respetes el formato de respuesta.
    """
context_s ="""
        Escribeme un resulen de este texto:
    """
context = [context_e, context_s]
info_extraction = {}
for key, value in clusters["max_polling_std"].items():
    texts = list(value["tweet_text"])
    t = ""
    info_extraction[key] = {
        "resumen": "",
        "eventos": "",
    }
    for text in texts:
        # info_extraction[key] = {
        #     "tipo evento": [],
        #     "lugar": [],
        #     "consecuencias": [],
        # }
        t += str(text)
    info_extraction[key]["resumen"] = ei(context_s, t).choices[0]["text"]
    info_extraction[key]["eventos"] = ei(context_e, t).choices[0]["text"]


In [26]:
info_extraction

{'cluster0': {'resumen': '\n\nEste texto refleja la preocupación de un usuario por la falta de solución a sus problemas de compra, bloqueo de cuenta, envío de productos, devolución, validación de información,',
  'eventos': '\n\ndict_eventos = {\n    "tipo evento": "Violencia a nivel nacional",\n    "lugar": "Chile",\n    "consecuencias": "Agresión a Carabineros",\n}\n\nprint(dict'},
 'cluster1': {'resumen': '\n\nEl resumen de este texto es que se necesita ayuda y soporte.',
  'eventos': '\n    No hay una respuesta apropiada para esta pregunta. Por favor, intente rephrasing su pregunta o proporcione más detalles.'},
 'cluster2': {'resumen': '\n\nEste texto trata sobre una situación en la que un pedido fue comprado y cobrado, pero no llegó nada. La persona solicita ayuda para actualizar una dirección y ofrece envío gratis, ad',
  'eventos': '\n\n{\n    "tipo_evento": "Comprado cobrado pero no llegó nada",\n    "lugar": "",\n    "consecuencias": "No puede actualizar una dirección, busca 

: 